
<p style="text-align:right; margin-bottom:0; padding-bottom:0; "></p><h2 style="margin : 0; padding-top:0;color:Orange;font-size:300%;"> Market Breadth Correlation Analysis<br></h2>

<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong>Importing Python Libraries</strong></span></span></p>


In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import bql

import scipy
from scipy.stats.mstats import winsorize

import plotly.express as px
import plotly.graph_objects as go

In [2]:
bq = bql.Service()

<p><strong><span style="color: orange; font-size: 24px;">1. Getting datasets via BQL</span></strong></p>

<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong>Downloading Custom Technicals</strong></span></span></p>


In [3]:
#Select universe and data items to download
index= 'SPX Index'
startdt = '2024-01-03' #yyyy-mm-dd
enddt = '2024-02-01' #yyyy-mm-dd

univ = bq.univ.members(index,dates=startdt)
price = {
    '50_SMAVG_Differential':
                            bq.func.sum(
                                bq.func.group( 
                                    bq.func.if_(
                                        bq.data.px_last(fill='prev',dates=startdt) - bq.data.SMAVG(close = bq.data.px_last(ca_adj='full'),fill='prev',period=50,per='d',dates=startdt) 
                                        > 0,1,0)
                                    )   
                                ) 
                        /bq.data.id().group().count(),

    }

request = bql.Request(univ, price)
response = bq.execute(request)
pd.concat([response.get(k).df()[[k]] for k in price.keys()], axis=1).head()

,50_SMAVG_Differential
ID,
IdentityGroup,0.932406


<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong>Downloading more custom Technicals</strong></span></span></p>


In [4]:
#Select universe and data items to download
#Downloading CMCI, SMAVG and MLRS technicals 
index = 'SPX index'
startdt = '2024-01-03' #yyyy-mm-dd
enddt = '2024-02-01' #yyyy-mm-dd

univ = bq.univ.members(index,dates=startdt)
price = {
    '50_SMAVG_Differential':
                            bq.func.sum(
                                bq.func.group( 
                                    bq.func.if_(
                                        bq.data.px_last(fill='prev',dates=startdt) - bq.data.SMAVG(close = bq.data.px_last(ca_adj='full'),fill='prev',period=50,per='d',dates=startdt) 
                                            > 0,1,0)
                                    )   
                                ) 
                        /bq.data.id().group().count(),
    'MLRS':
                            bq.func.sum(
                                bq.func.group( 
                                    bq.func.if_(
                                        bq.data.MLRS(dates=startdt,Period=90,fill='prev',per='d') 
                                            > 0,1,0)
                                    )   
                                ) 
                        /bq.data.id().group().count(),

    'CMCI':
                    bq.func.sum(
                        bq.func.group( 
                            bq.func.if_(
                                bq.data.CMCI(dates=startdt,Period=14,fill='prev',per='d') 
                                    < 0,1,0)
                            )   
                        ) 
                /bq.data.id().group().count()
    }
request = bql.Request(univ, price)
response = bq.execute(request)
pd.concat([response.get(k).df()[[k]] for k in price.keys()], axis=1).head()

,50_SMAVG_Differential,MLRS,CMCI
ID,,,
IdentityGroup,0.932406,0.685885,0.586481


<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong> Downloading forward performance</strong></span></span></p>


In [5]:
#Select universe and data items to download
#Downloading CMCI, SMAVG and MLRS technicals + 1M forward index return 
index = 'SPX index'
startdt = '2024-01-03' #yyyy-mm-dd
enddt = '2024-02-01' #yyyy-mm-dd

univ = bq.univ.members(index,dates=startdt)
price = {
    'Date': bq.func.value(bq.data.total_return(calc_interval=bq.func.range(startdt,enddt)),bq.univ.list('SPX Index')).group().first()['DATE'],
    'Ret_1m': bq.func.value(bq.data.total_return(calc_interval=bq.func.range(startdt,enddt)),bq.univ.list('SPX Index')).group().first(),
    '50_SMAVG_Differential':
                            bq.func.sum(
                                bq.func.group( 
                                    bq.func.if_(
                                        bq.data.px_last(fill='prev',dates=startdt) - bq.data.SMAVG(close = bq.data.px_last(ca_adj='full'),fill='prev',period=50,per='d',dates=startdt) > 0,1,0
                                        )
                                    )   
                                ) 
                        /bq.data.id().group().count(),
    'MLRS':
                            bq.func.sum(
                                bq.func.group( 
                                    bq.func.if_(
                                        bq.data.MLRS(dates=startdt,Period=90,fill='prev',per='d') > 0,1,0
                                        )
                                    )   
                                ) 
                        /bq.data.id().group().count(),

    'CMCI':
                    bq.func.sum(
                        bq.func.group( 
                            bq.func.if_(
                                bq.data.CMCI(dates=startdt,Period=14,fill='prev',per='d') 
                                    < 0,1,0)
                            )   
                        ) 
                /bq.data.id().group().count()
    }
request = bql.Request(univ, price)
response = bq.execute(request)
pd.concat([response.get(k).df()[[k]] for k in price.keys()], axis=1).head()

,Date,Ret_1m,50_SMAVG_Differential,MLRS,CMCI
ID,,,,,
IdentityGroup,2024-02-01,0.043702,0.932406,0.685885,0.586481


<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong> Iterate across multiple dates</strong></span></span></p>


In [6]:
# Create a timeseries of date snapshots in the interval that we are interested in 
dt_range = pd.date_range(start='2015-01-01',end='2024-09-16',freq='2W')
dt_range

DatetimeIndex(['2015-01-04', '2015-01-18', '2015-02-01', '2015-02-15',
               '2015-03-01', '2015-03-15', '2015-03-29', '2015-04-12',
               '2015-04-26', '2015-05-10',
               ...
               '2024-05-12', '2024-05-26', '2024-06-09', '2024-06-23',
               '2024-07-07', '2024-07-21', '2024-08-04', '2024-08-18',
               '2024-09-01', '2024-09-15'],
              dtype='datetime64[ns]', length=254, freq='2W-SUN')

In [8]:
# creating two containers to store all results and the BQL queries 
results = []
request_package = []

#creating a list of tuples with two dates that represent start and end of the analysis window
list_of_dates = list(zip(dt_range[:-1],dt_range[1:]))


#Select universe and data items to download
index = 'SPX Index'
for dt in list_of_dates:
    startdt = dt[0].strftime('%Y-%m-%d')
    enddt = dt[1].strftime('%Y-%m-%d')
    
    univ = bq.univ.members(index,dates=startdt)
    price = {
        'Date': bq.func.value(bq.data.total_return(calc_interval=bq.func.range(startdt,enddt)),bq.univ.list(index)).group().first()['DATE'],
        'Ret_1m': bq.func.value(bq.data.total_return(calc_interval=bq.func.range(startdt,enddt)),bq.univ.list(index)).group().first(),

        '50_SMAVG_Differential':
                                bq.func.sum(
                                    bq.func.group( 
                                        bq.func.if_(
                                            bq.data.px_last(fill='prev',dates=startdt) - bq.data.SMAVG(close = bq.data.px_last(ca_adj='full'),fill='prev',period=50,per='d',dates=startdt) > 0,1,0
                                            )
                                        )   
                                    ) 
                            /bq.data.id().group().count(),
        'MLRS':
                                bq.func.sum(
                                    bq.func.group( 
                                        bq.func.if_(
                                            bq.data.MLRS(dates=startdt,Period=90,fill='prev',per='d') > 0,1,0
                                            )
                                        )   
                                    ) 
                            /bq.data.id().group().count(),
        
        'CMCI':
                        bq.func.sum(
                            bq.func.group( 
                                bq.func.if_(
                                    bq.data.CMCI(dates=startdt,Period=14,fill='prev',per='d') 
                                        < 0,1,0)
                                )   
                            ) 
                    /bq.data.id().group().count()
        }
    request = bql.Request(univ, price)
    
    # append all BQL requests in the request_package list 
    request_package.append(request)

<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong> Using bq.execute_many to execute all bql queries in list</strong></span></span></p>



In [ ]:
# using bql's execute_many method to execute multiple BQL calls all at once  
response = bq.execute_many(request_package)
data = []
for i,r in enumerate(response):
    data.append(pd.concat([r.get(k).df().reset_index()[[k]] for k in price.keys()], axis=1).dropna())  # store all results in data list   

    
#concat all data in list into a final dataframe 
df_executemany = pd.concat(data).set_index('Date') 


<p><span style="text-align: inherit;"><span style="font-size: 20px;"><strong> Preserve original dataframe & make a copy</strong></span></span></p>


In [ ]:
#making a new copy of the dataframe and save it in df_
df_ = df_executemany
df_.head()

<p><strong><span style="color: orange; font-size: 24px;">2. Exploratory Data Analysis - Spotting possible patterns</span></strong></p>

In [ ]:
# Chart historical market breadth over time using Plotly
fig = go.Figure()

# Add traces for MLRS technicals 
fig.add_trace(go.Scatter(x=df_.index, y=df_['MLRS']))

fig.update_layout(template='plotly_dark',showlegend=False,height=400,title='Historical Market Breadth (MLRS)')
fig.show()

In [ ]:
# Chart historical market breadth over time using Plotly
fig = go.Figure()

# Add traces for CMCI technicals
fig.add_trace(go.Scatter(x=df_.index, y=df_['CMCI']))

fig.update_layout(template='plotly_dark',showlegend=False,height=400,title='Historical Market Breadth (CMCI)')
fig.show()

In [ ]:
# Visualize distribution of datasets from BQL query 
# Further transformations might be needed as data distribution is not standardized 

fig = go.Figure()
for i in df_.columns:
    y0 = df_[i]
    fig.add_trace(go.Box(y=y0,name=i))

fig.update_traces(boxpoints='all', jitter=0)
fig.update_layout(template='plotly_dark',showlegend=False,height=400,title='Historical Market Breadth (CMCI)')


In [ ]:
# apply zscores & winsorize methods to normalize dataframe 
_ = df_
for col in _.columns: 
     _[col] = winsorize(_[col], limits=0.05)

_ = df_.apply(scipy.stats.zscore)

# Visualize new distribution of data after normalization 
fig = go.Figure()
for i in _.columns:
    y0 = _[i]
    fig.add_trace(go.Box(y=y0,name=i))

fig.update_traces(boxpoints='all', jitter=0)
fig.update_layout(template='plotly_dark',showlegend=False,height=400,title='Historical Market Breadth (CMCI)')


In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=_.index, y=_['Ret_1m'],name = 'Ret'))
fig.add_trace(go.Scatter(x=_.index, y=_['MLRS'],name = 'MLRS'))

fig.update_layout(template='plotly_dark',showlegend=False,height=400,title='Historical Market Breadth')
fig.show()

In [ ]:
_

In [ ]:
np.corrcoef(_['Ret_1m'][-15:], _['MLRS'][-15:])

In [ ]:
_['rollcorr_MLRS'] = _['MLRS'].rolling(15).corr(_['Ret_1m'])
_['rollcorr_CMCI'] = _['CMCI'].rolling(15).corr(_['Ret_1m'])
_['rollcorr_SMA'] = _['50_SMAVG_Differential'].rolling(15).corr(_['Ret_1m'])

In [ ]:
_.dropna().tail()

In [ ]:
px.line(_[['rollcorr_MLRS','rollcorr_CMCI','rollcorr_SMA']])